# Вивчення асоціативних правил з використанням бібліотеки mlxtend

## Мета:
Навчитися застосовувати алгоритми пошуку асоціативних правил для аналізу даних з використанням бібліотеки mlxtend у Python.

## Кроки виконання:

### Завантаження та підготовка даних:

- Завантажте набір даних з Kaggle.
- Виберіть потрібні стовпці та видаліть порожні значення.
- Переконайтеся, що кількість придбаних товарів більша за 0.
- Перетворіть дані у формат, придатний для алгоритму Apriori (матриця, де рядки – це покупки, а стовпці – товари, значення – наявність товару в покупці).

### 

In [1]:
import pandas as pd

In [2]:
retail = pd.read_excel('datasets/online_retail_II.xlsx')

retail

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [3]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


### Застосування алгоритму Apriori:

- Використайте функцію apriori для пошуку частих наборів елементів.
- Встановіть мінімальну підтримку (min_support) на рівні, який дозволить знайти цікаві часті набори елементів.

### Побудова асоціативних правил:
- Використайте функцію association_rules для побудови асоціативних правил.
- Виберіть метрику (наприклад, lift) та встановіть мінімальний поріг для цієї метрики.

### Аналіз результатів:
- Проаналізуйте отримані часті набори елементів та асоціативні правила.
- Інтерпретуйте результати: які товари часто купуються разом, які правила можуть бути корисними для маркетингових стратегій.

## Завдання для самостійного аналізу:


### Визначення оптимальних параметрів:

Експериментуйте з різними значеннями мінімальної підтримки (min_support) та пороговими значеннями для метрики lift. Визначте, які параметри дозволяють отримати найкорисніші та найцікавіші правила.

### Візуалізація результатів:

Використайте бібліотеки для візуалізації частих наборів елементів та асоціативних правил. Спробуйте створити графіки, які наочно покажуть зв'язки між товарами.

### Аналіз конкретних правил:

Виберіть декілька найцікавіших асоціативних правил та детально проаналізуйте їх. Розгляньте можливі причини, чому ці правила є корисними для бізнесу, та як їх можна застосувати на практиці.

### Порівняння з іншими алгоритмами:

Досліджуйте інші алгоритми для пошуку асоціативних правил (наприклад, Eclat) та порівняйте їх результати з алгоритмом Apriori. Оцініть переваги та недоліки кожного з них.

### Застосування до іншого набору даних:

Спробуйте застосувати вивчені методи до іншого медичного набору даних з Kaggle. Порівняйте результати та зробіть висновки щодо універсальності методів та алгоритмів.